In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
page = requests.get('http://advisory.mtanyct.info/eedevwebsvc/allequipments.aspx')

In [3]:
soup = BeautifulSoup(page.content, 'lxml')

In [4]:
station = [t.text for t in soup.findAll('station')]
el_id = [t.text for t in soup.findAll('equipmentno')]
desc = [t.text for t in soup.findAll('serving')]
borough = [t.text for t in soup.findAll('borough')]
lines = [t.text for t in soup.findAll('trainno')]
equipment_type = [t.text for t in soup.findAll('equipmenttype')]
ada = [t.text for t in soup.findAll('ada')]
is_active = [t.text for t in soup.findAll('isactive')]


In [5]:
elevators_master_list = pd.DataFrame(list(zip(station,el_id,desc,borough,lines,equipment_type, ada, is_active)),
                                     columns=["station_name","equipment_id","description","borough","subway_lines",
                                             "equipment_type","ada_compliant","is_active"])

In [6]:
elevators_master_list.head(20)

,station_name,equipment_id,description,borough,subway_lines,equipment_type,ada_compliant,is_active
0,125 St,EL125,125 St & Lexington Ave (NE corner) to Mezzanin...,MN,4/5/6/METRO-NORTH,EL,Y,Y
1,125 St,EL142,125 St & St Nicholas Ave (SW corner) to Mezzan...,MN,A/B/C/D,EL,Y,Y
2,125 St,EL143,Mezzanine to downtown Platform,MN,A/B/C/D,EL,Y,Y
3,125 St,EL126,Mezzanine to Platforms for service in both dir...,MN,4/5/6/METRO-NORTH,EL,Y,Y
4,125 St,EL144,Mezzanine to uptown Platform,MN,A/B/C/D,EL,Y,Y
5,125 St,ES102,Street to Mezzanine,MN,1,ES,N,Y
6,125 St,ES101,Street to Mezzanine,MN,1,ES,N,Y
7,125 St,ES103,Street to Mezzanine,MN,1,ES,N,Y
8,135 St,EL180,Lenox Ave & 181 St (NE corner) to uptown Platform,MN,2/3,EL,Y,Y
9,135 St,EL181,Lenox Ave & 181 St (SW corner) to downtown Pla...,MN,2/3,EL,Y,Y


In [7]:
list(elevators_master_list.description)

['125 St & Lexington Ave (NE corner) to Mezzanine for service in both directions',
 '125 St & St Nicholas Ave (SW corner) to Mezzanine for service in both directions',
 'Mezzanine to downtown Platform',
 'Mezzanine to Platforms for service in both directions on all lines',
 'Mezzanine to uptown Platform',
 'Street to Mezzanine',
 'Street to Mezzanine',
 'Street to Mezzanine',
 'Lenox Ave & 181 St (NE corner) to uptown Platform',
 'Lenox Ave & 181 St (SW corner) to downtown Platform',
 '8th Ave & 14 St (SW corner) to Mezzanine for service in both directions',
 'Mezzanine to downtown A/C/E Platform',
 'Mezzanine to uptown A/C/E Platform',
 '4th Ave & 14 St (NE corner) to Mezzanine for service in both directions',
 'Mezzanine to downtown N/Q/R/W Platform',
 'Mezzanine to Platform for service in both directions',
 'Mezzanine to uptown N/Q/R/W Platform',
 'Street to Mezzanine all trains',
 'Street to Mezzanine all trains',
 'Three landings',
 'Northbound ll Platform to Upper Mezzanine',
 'N

TODO: split lines to make data more long-form

TODO: parse descriptions to create columns 
- subset of connecting lines
- if connection to street level
- direction of travel


In [8]:
def get_direction(x):
    r = re.findall('uptown|downtown|both directions', x)
    if len(r):
        return r[0]
    else:
        return ""

In [9]:
elevators_master_list['direction'] = [get_direction(x) for x in elevators_master_list.description]

In [10]:
tmp = [re.findall(y,x) for x,y in zip(elevators_master_list.description,elevators_master_list.subway_lines)]
elevators_master_list['subset_lines'] = [x if x else y for x,y in zip(tmp,elevators_master_list.subway_lines)]

In [11]:
tmp = [re.match('Street|St|Ave|{}'.format(y),x) for x,y in zip(elevators_master_list.description,elevators_master_list.station_name)]
elevators_master_list['connection_to_street'] =["Y" if x else "N" for x in tmp]

In [12]:
elevators_master_list.to_csv('EE_master_list.csv')